In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import tqdm


def get_best_authors_per_sector(df, start_date, end_date, max_authors=10):
    """
    Find the best performing authors per sector based on correlation between
    expected and actual returns within the specified date range.
    """
    # Filter data for the specified period
    mask = (df['date'] >= start_date) & (df['date'] <= end_date)
    period_df = df[mask].copy()
    
    # Calculate correlation for each author-sector pair
    author_sector_performance = []
    
    for (sector, author), group in period_df.groupby(['Sector', 'author']):
        if len(group) < 2:  # Skip if not enough data points
            continue
            
        correlation = np.corrcoef(group['expected_return'], group['actual_return'])[0, 1]
        
        if not np.isnan(correlation) and correlation > 0:
            author_sector_performance.append({
                'Sector': sector,
                'author': author,
                'correlation': correlation
            })
    
    # Convert to DataFrame and get top authors per sector
    performance_df = pd.DataFrame(author_sector_performance)
    if len(performance_df) == 0:
        return pd.DataFrame(columns=['Sector', 'author', 'correlation'])
        
    return (performance_df.sort_values('correlation', ascending=False)
            .groupby('Sector')
            .head(max_authors)
            .reset_index(drop=True))

def calculate_monthly_performance(df, best_authors_df, target_month):
    """
    Calculate performance for each author-sector pair for a specific month
    using sign(expected_return) * actual_return
    """
    # Filter data for the target month and best authors
    month_start = target_month.replace(day=1)
    month_end = (month_start + pd.offsets.MonthEnd(1))
    
    mask = (df['date'] >= month_start) & (df['date'] <= month_end)
    month_df = df[mask].copy()
    
    # Filter for only the best author-sector pairs
    month_df = month_df.merge(
        best_authors_df[['Sector', 'author']],
        on=['Sector', 'author'],
        how='inner'
    )
    
    # Calculate performance
    month_df['performance'] = np.sign(month_df['expected_return']) * month_df['actual_return']

    # Group by Sector and Author to calculate correlation and performance sum
    performance_summary = month_df.groupby(['Sector', 'author']).agg(
        correlation=('expected_return', lambda x: np.corrcoef(
            x, 
            month_df.loc[x.index, 'actual_return']
        )[0, 1] if len(x) > 1 else 0),  # Avoid errors with single data points
        sum_performance=('performance', 'sum'),
        count=('performance', 'count')
    ).reset_index()
    
    # Compute logical AND (&&) of correlation and sum_performance
    # Logical AND: True (1) if both are non-zero, False (0) otherwise
    performance_summary['and_correlation_sum'] = (
        (performance_summary['correlation'] >= 0) & (performance_summary['sum_performance'] >= 0)
    ).astype(int)  # Convert to 1 (True) or 0 (False)

    # Add the month column for reference
    performance_summary['month'] = month_start

    return performance_summary

def run_rolling_analysis(df, start_date, end_date, lookback_period=12):
    """
    Run the rolling analysis month by month
    """
    # Convert dates if they're strings
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    all_monthly_performance = []
    
    current_date = start_date
    while current_date <= end_date:
        # Calculate training period
        training_end = current_date - timedelta(days=1)
        training_start = training_end - pd.DateOffset(months=lookback_period)
        
        # Get best authors based on the training period
        best_authors = get_best_authors_per_sector(
            df,
            training_start,
            training_end
        )
        
        # Calculate performance for the current month
        if not best_authors.empty:
            monthly_perf = calculate_monthly_performance(
                df,
                best_authors,
                current_date
            )
            all_monthly_performance.append(monthly_perf)
        
        # Move to next month
        current_date = current_date + pd.DateOffset(months=1)
    
    # Combine all results
    if all_monthly_performance:
        return pd.concat(all_monthly_performance, ignore_index=True)
    return pd.DataFrame()

def print_monthly_sector_author_performance(results_df):
    """
    Print formatted results showing performance by month, sector, and author
    """
    # Sort by month, sector, and mean performance
    sorted_results = results_df.sort_values(
        ['month', 'Sector', 'and_correlation_sum'],
        ascending=[True, True, False]
    )
    
    # Format and print results
    current_month = None
    current_sector = None
    
    for _, row in sorted_results.iterrows():
        month_str = row['month'].strftime('%Y-%m')
        
        if month_str != current_month:
            print(f"\n=== Month: {month_str} ===")
            current_month = month_str
            current_sector = None
        
        if row['Sector'] != current_sector:
            print(f"\nSector: {row['Sector']}")
            current_sector = row['Sector']
        
        print(f"  Author: {row['author']:<30} "
              f"Correlation AND Sum of Performance: {row['and_correlation_sum']:6.3f} "
              f"Number of Predictions: {row['count']:3d}")

# Example usage
def main(df):
    # Convert date column to datetime if it's not already
    df['date'] = pd.to_datetime(df['date'])
    
    # Run the analysis
    results = run_rolling_analysis(
        df,
        start_date='2014-06-01',
        end_date='2024-05-31',
        lookback_period=12
    )
    
    # Print results
    print_monthly_sector_author_performance(results)
    
    return results

In [8]:
# Load the filtered DataFrame from the CSV file
# Define the file path
file_path = r'C:\Users\disch\Desktop\CiteSert\Project_1\Version2_40krows_n5\input_data\filtered_data_2.csv'

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(file_path)

df = df.dropna(subset=['expected_return', 'actual_return'])


# Display the first few rows of the DataFrame to verify it loaded correctly
print(df.head())

          date                    Company Name      Symbol  \
0   2015-11-16  speciality restaurants limited  SPECIALITY   
1   2015-08-13  speciality restaurants limited  SPECIALITY   
8   2024-05-17           eicher motors limited   EICHERMOT   
9   2024-05-13           eicher motors limited   EICHERMOT   
11  2024-05-13           eicher motors limited   EICHERMOT   

                      author         Sector  expected_return  actual_return  
0                      Karvy     Technology         9.432421      -0.018033  
1        Reliance Securities     Technology         7.189542       0.018065  
8   BOB Capital Markets Ltd.  Manufacturing        -1.297429       0.030817  
9               ICICI Direct  Manufacturing        18.090371       0.001189  
11               Axis Direct  Manufacturing        10.253465       0.001189  


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 35107 entries, 0 to 41819
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             35107 non-null  object 
 1   Company Name     35107 non-null  object 
 2   Symbol           35107 non-null  object 
 3   author           35107 non-null  object 
 4   Sector           35107 non-null  object 
 5   expected_return  35107 non-null  float64
 6   actual_return    35107 non-null  float64
dtypes: float64(2), object(5)
memory usage: 2.1+ MB


In [10]:
df2 = main(df)

C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
C:\ProgramData\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: in


=== Month: 2014-06 ===

Sector: Consumer Goods
  Author: Chola Wealth Direct            Correlation AND Sum of Performance:  0.000 Number of Predictions:   1

Sector: Energy
  Author: Chola Wealth Direct            Correlation AND Sum of Performance:  1.000 Number of Predictions:   1

Sector: Finance
  Author: Chola Wealth Direct            Correlation AND Sum of Performance:  1.000 Number of Predictions:   1

Sector: Healthcare
  Author: Chola Wealth Direct            Correlation AND Sum of Performance:  1.000 Number of Predictions:   1

=== Month: 2014-07 ===

Sector: Finance
  Author: Chola Wealth Direct            Correlation AND Sum of Performance:  0.000 Number of Predictions:   7

Sector: Manufacturing
  Author: Chola Wealth Direct            Correlation AND Sum of Performance:  0.000 Number of Predictions:   3

Sector: Technology
  Author: Chola Wealth Direct            Correlation AND Sum of Performance:  1.000 Number of Predictions:   1

=== Month: 2014-08 ===

Sector: Consu

In [11]:
df2.describe()

,correlation,sum_performance,count,and_correlation_sum,month
count,3008.000000,3029.000000,3029.000000,3029.000000,3029
mean,-0.008285,0.007676,3.725982,0.405084,2020-02-13 15:37:29.851436288
min,-1.000000,-10.032209,1.000000,0.000000,2014-06-01 00:00:00
25%,-0.116685,-0.038762,1.000000,0.000000,2018-01-01 00:00:00
50%,0.000000,0.003709,2.000000,0.000000,2020-02-01 00:00:00
75%,0.116251,0.047160,4.000000,1.000000,2022-06-01 00:00:00
max,1.000000,4.856564,42.000000,1.000000,2024-05-01 00:00:00
std,0.538470,0.247874,5.012882,0.490989,NaN
